Cuando busquéis información geográfica podréis encontrarla en muchos formatos: shapefile, kmz, geojson, topojson,... 
Para hacer representaciones en folium necesitamos formato json (geojson o topojson). Las diferencias entre estas dos son que los topojson tienen mayor grado de compresión, elimina la redundancia y usa una codificación de coordenadas de enteros de precisión fija más eficiente que hace que un archivo topoJSON pueda llegar a ser un 80% menor que un geojson para una misma región geográfica. Sin embargo, a cambio, los topoJSON presentan un formato de archivo bastante más complejo que hace más dificil entender su estructura.

Para obtener los geojson necesarios para nuestras representaciones en folium podemos hacer varias cosas:

* Generar nosotros mismos los geojson: http://geojson.io

* Descargar shape files (muy fáciles de encontrar a variados niveles de fronteras administrativas por ejemplo en https://gadm.org/) y convertir a geojson:
Usando la librería de python pyshp
En la web: http://mapshaper.org/
* Descargar geojson de diversas fuentes que podéis ir encontrando, por ejemplo:
    * Países del mundo : https://github.com/johan/world.geo.json/tree/master/countries
    * Autonomías/provincias de España : https://github.com/codeforamerica/click_that_hood/tree/master/public/data
    * Códigos postales España: https://github.com/inigoflores/ds-codigos-postales/tree/master/data
    * En páginas propias del estado y regiones: 
http://datos.gob.es, https://datos.madrid.es/, https://datosabiertos.malaga.eu/


In [1]:
import json
import pandas as pd
import folium

Tenemos dos archivos para este primer ejercicio: un geojson que define la geometría de los estados de USA y un csv con la información de desempleo que queremos representar en este mapa.

In [15]:
infoUnemployment = pd.read_csv("../data/US_Unemployment_Oct2012.csv")
infoUnemployment.head(4)

,State,Unemployment
0,AL,7.1
1,AK,6.8
2,AZ,8.1
3,AR,7.2


Te recomiendo dedicar unos minutos a entender la información contenida en el geojson.

In [3]:
us_statesjson = "../data/us-states.json"
geo_us_state = json.load(open(us_statesjson))
geo_us_state

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'id': 'AL',
   'properties': {'name': 'Alabama'},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-87.359296, 35.00118],
      [-85.606675, 34.984749],
      [-85.431413, 34.124869],
      [-85.184951, 32.859696],
      [-85.069935, 32.580372],
      [-84.960397, 32.421541],
      [-85.004212, 32.322956],
      [-84.889196, 32.262709],
      [-85.058981, 32.13674],
      [-85.053504, 32.01077],
      [-85.141136, 31.840985],
      [-85.042551, 31.539753],
      [-85.113751, 31.27686],
      [-85.004212, 31.003013],
      [-85.497137, 30.997536],
      [-87.600282, 30.997536],
      [-87.633143, 30.86609],
      [-87.408589, 30.674397],
      [-87.446927, 30.510088],
      [-87.37025, 30.427934],
      [-87.518128, 30.280057],
      [-87.655051, 30.247195],
      [-87.90699, 30.411504],
      [-87.934375, 30.657966],
      [-88.011052, 30.685351],
      [-88.10416, 30.499135],
      [-88.137022, 30.318396],
    

In [4]:
geo_us_state.keys()

dict_keys(['type', 'features'])

In [5]:
type(geo_us_state["features"])

list

In [6]:
len(geo_us_state["features"])

50

In [7]:
geo_us_state["features"][0]

{'type': 'Feature',
 'id': 'AL',
 'properties': {'name': 'Alabama'},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-87.359296, 35.00118],
    [-85.606675, 34.984749],
    [-85.431413, 34.124869],
    [-85.184951, 32.859696],
    [-85.069935, 32.580372],
    [-84.960397, 32.421541],
    [-85.004212, 32.322956],
    [-84.889196, 32.262709],
    [-85.058981, 32.13674],
    [-85.053504, 32.01077],
    [-85.141136, 31.840985],
    [-85.042551, 31.539753],
    [-85.113751, 31.27686],
    [-85.004212, 31.003013],
    [-85.497137, 30.997536],
    [-87.600282, 30.997536],
    [-87.633143, 30.86609],
    [-87.408589, 30.674397],
    [-87.446927, 30.510088],
    [-87.37025, 30.427934],
    [-87.518128, 30.280057],
    [-87.655051, 30.247195],
    [-87.90699, 30.411504],
    [-87.934375, 30.657966],
    [-88.011052, 30.685351],
    [-88.10416, 30.499135],
    [-88.137022, 30.318396],
    [-88.394438, 30.367688],
    [-88.471115, 31.895754],
    [-88.241084, 33.796253],
    [-88.098683, 34.8

Vemos que la manera de "unir" ambos orígenes de datos será mediante el id de cada feature del geojson y el campo State del df con datos numéricos.

In [16]:
geo_us_state["features"][0]["id"]

'AL'

### 1- Representación de una geometría 

Empezaremos representando la geometría del geojson, correspondiente a los estados de USA. 

In [9]:
# 0- Entender la estructura de los datos que tengo...

# 1- Iniciamos el mapa
m = folium.Map([43,-100], zoom_start=4)

# 2- Creamos las funciones que van a definir el estilo de cada área

funcionEstilo = lambda feature: {
            'fillColor': "red", # interior
            'color' : 'black', # linea
            'fillOpacity': 0.1, # interior
            'opacity': 1, # línea
            'weight' : 1, # línea
                          }
funcionResalte =  lambda feature: {
            'fillColor': "red", # interior
            'color' : 'red', # linea
            'fillOpacity': 0.5, # interior
            'opacity': 1, # línea
            'weight' : 1, # línea
                          }

# 3- Añadimos cada área de mi json, customizando colores, opacidades,...definiendo una función
# que se aplica sobre cada feature de mi geojson (sobre cada estado de USA en este caso)
USAgeom= folium.GeoJson(
        data = geo_us_state#geo_json_us_states#us_states # URL, file path, or data (json, dict, geopandas, etc) to your GeoJSON geometries
        ,style_function=funcionEstilo,
        highlight_function=funcionResalte
        )

# 4- Añado geometría al mapa
USAgeom.add_to(m)
m

Imaginad que hay un estado en concreto que quiero que aparezca de otro color por ejemplo:

In [13]:
# mapa vacío
m = folium.Map(location = [43,-100],zoom_start=4)

# defino estilos de las geometrías -> defines el interior distinto según el nombre de cada feature.
# si es Alabama será otro color diferente
funcionEstilo = lambda x: {
            'fillColor': "black" if x["properties"]["name"] =="Alabama" else "orange" , # interior
            'color' : 'black', # linea
            'fillOpacity': 0.4, # interior
            'opacity': 1, # línea
            'weight' : 1, # línea
                          }
funcionResalte =  lambda x: {
            'fillColor': "blue", # interior
            'color' : 'red', # linea
            'fillOpacity': 0.5, # interior
            'opacity': 1, # línea
            'weight' : 1, # línea
                          }
 
# mapa con geometrías
geomUSA = folium.GeoJson(data = geo_us_state,
                        style_function = funcionEstilo
                        , highlight_function = funcionResalte)
# añado 
geomUSA.add_to(m)
m

### 2- Descubre las paletas de colores

Para que los colores de nuestra geometría varíen en función de un valor numéricos vamos a aprender unas pinceladas de los mapas de colores.

In [17]:
import branca.colormap as cm

In [18]:
cm.LinearColormap(colors=["red","blue"],vmin=0,vmax=10)

In [19]:
cm.StepColormap(colors=["blue","yellow","red"],vmin=0,vmax=10,index=[0,4,6])

In [22]:
a = cm.linear.YlGnBu_09.scale(vmin=0,vmax = 20)
a

In [23]:
print(a(19.8))
print(a(20))
print(a(4555))
print(a(0))
print(a(-7))

#0a1e5d
#081d58
#081d58
#ffffd9
#ffffd9


### 3- Representación de una geometría con variación de colores

Convierto mim df con info numérica en una serie cuyo índice sea el estado.

In [26]:
datausaserie = infoUnemployment.set_index("State")["Unemployment"]
datausaserie

State
AL     7.1
AK     6.8
AZ     8.1
AR     7.2
CA    10.1
CO     7.7
CT     8.4
DE     7.1
FL     8.2
GA     8.8
HI     5.4
ID     6.6
IL     8.8
IN     8.4
IA     5.1
KS     5.6
KY     8.1
LA     5.9
ME     7.2
MD     6.8
MA     6.7
MI     9.1
MN     5.6
MS     9.1
MO     6.7
MT     5.8
NE     3.9
NV    10.3
NH     5.7
NJ     9.6
NM     6.8
NY     8.4
NC     9.4
ND     3.2
OH     6.9
OK     5.2
OR     8.5
PA     8.0
RI    10.1
SC     8.8
SD     4.4
TN     7.8
TX     6.4
UT     5.5
VT     5.0
VA     5.8
WA     7.8
WV     7.5
WI     6.8
WY     5.1
Name: Unemployment, dtype: float64

In [29]:
escala = cm.linear.YlGnBu_09.scale(vmin=0,vmax=max(datausaserie))
escala

In [30]:
print(datausaserie["AL"])
escala(datausaserie["AL"])

7.1


'#1f77b4'

In [32]:
# 0- Entender la estructura de los datos que tengo...y adaptar los datos como sea necesario..
# Genero una pandas serie, en la que los índices sea la key que me liga los datos con el geojon
infoUnemploymentSerie = infoUnemployment.set_index('State')['Unemployment']

# 1- Iniciamos el mapa
m = folium.Map([43,-100], zoom_start=4)

# 1- Genero la escala de colores que voy a usar
escala = cm.linear.YlOrRd_09.scale(vmin=0, vmax= max(datausaserie))
escala.caption="Unemployment rate"

# 2- Creamos las funciones que van a definir el estilo de cada área
funcionEstilo = lambda feature: {
            'fillColor': escala(datausaserie[feature['id']]), # interior
            'color' : 'black', # linea
            'fillOpacity': 0.5, # interior
            'opacity': 1, # línea
            'weight' : 1, # línea         
                      }  


# 3- Añadimos cada área de mi json, customizando colores, opacidades,...definiendo una función
# que se aplica sobre cada feature de mi geojson (sobre cada estado de USA en este caso)
USAgeom= folium.GeoJson(
        data = geo_us_state#geo_json_us_states#us_states # URL, file path, or data (json, dict, geopandas, etc) to your GeoJSON geometries
        ,style_function=funcionEstilo
        )

# 4- Añado geometría al mapa
USAgeom.add_to(m)

# 5- Le añado la leyenda
m.add_child(escala)

m

##### EJERCICIO REPRESENTA LA POBLACIÓN POR DISTRITO DE MÁLAGA SEGÚN LA INFORMACIÓN OBTENIDA DE OPENDATA MÁLAGA

In [33]:
info=pd.read_csv("../data/padrondistritosmunicipales.csv")#("http://datosabiertos.malaga.eu/recursos/demografia/padron/2017/padrondistritosmunicipales.csv")
info.head(4)
# CPRON, CMUNN códigos de provincia y municipio de nacimiento
# NACI código de la nacionalidad 108 es nacionalidad española
# DTOMUN identificador del distrito 

,NHOP,EDAD,SEXO,CPRON,CMUNN,NACI,DTOMUN,ID
0,1329,3,6,29,67,108,8,1
1,1225,8,6,29,67,108,8,2
2,1225,5,6,29,67,108,8,3
3,1225,39,1,29,67,108,8,4


 Quiero representar el total de población para cada distrito, por lo que lo primero será calcular el volumen de población para cada distrito.

In [34]:
infoPoblacion = info.groupby(by="DTOMUN")["CMUNN"].count()
infoPoblacion

DTOMUN
1      80673
2      56108
3      36212
4      60310
5      30378
6      85617
7     114664
8      19622
9      18678
10     30524
11     37220
Name: CMUNN, dtype: int64

In [35]:
malagadistritos = json.load(open("../data/da_cartografiaDistritoMunicipal-4326.geojson"))
malagadistritos

{'type': 'FeatureCollection',
 'totalFeatures': 11,
 'features': [{'type': 'Feature',
   'id': 'da_cartografiaDistritoMunicipal.91',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-4.51628402, 36.69604594, 3.735e-05],
      [-4.51625898, 36.69599894, 3.735e-05],
      [-4.51627541, 36.69589507, 3.735e-05],
      [-4.51629283, 36.69582724, 3.735e-05],
      [-4.51631024, 36.69575941, 3.735e-05],
      [-4.51651387, 36.69508754, 3.735e-05],
      [-4.51655184, 36.69491804, 3.735e-05],
      [-4.51658636, 36.69478915, 3.735e-05],
      [-4.51662703, 36.69470526, 3.735e-05],
      [-4.51667946, 36.69462797, 3.735e-05],
      [-4.51676315, 36.69451198, 3.735e-05],
      [-4.51687623, 36.69437082, 3.735e-05],
      [-4.51692559, 36.69430259, 3.735e-05],
      [-4.51693999, 36.69425508, 3.735e-05],
      [-4.5169458, 36.69419191, 3.735e-05],
      [-4.51695753, 36.69411063, 3.735e-05],
      [-4.51694611, 36.6939767, 3.735e-05],
      [-4.51693469, 36.69384276, 3.735e-05],
      [-4

In [36]:
# 0- Entender la estructura de los datos que tengo...y adaptar los datos como sea necesario..

# 1- Iniciamos el mapa
m = folium.Map(location = [36.715090, -4.461963],zoom_start= 11)

# 1- Genero la escala de colores que voy a usar
escala = cm.linear.BuPu_07.scale(vmin=0, vmax= max(infoPoblacion))
escala.caption="Población"

# 2- Creamos las funciones que van a definir el estilo de cada área
funcionEstilo = lambda feature: {
            'fillColor': escala(infoPoblacion[feature["properties"]["NUMERO"]]), # interior
            'color' : 'black', # linea
            'fillOpacity': 0.5, # interior
            'opacity': 1, # línea
            'weight' : 1, # línea
            'dashArray' : "1 2"
                      }  
funcionResalte =  lambda feature: {
            'fillColor': escala(infoPoblacion[feature["properties"]["NUMERO"]]), # interior
            'color' : 'black', # linea
            'fillOpacity': 0.5, # interior
            'opacity': 1, # línea
            'weight' : 2, # línea
            'dashArray' : "0"
                          }

# 3- Añadimos cada área de mi json, customizando colores, opacidades,...definiendo una función
# que se aplica sobre cada feature de mi geojson (sobre cada estado de USA en este caso)
Malagageompobla= folium.GeoJson(
        data = malagadistritos# URL, file path, or data (json, dict, geopandas, etc) to your GeoJSON geometries
        ,style_function=funcionEstilo,
        highlight_function=funcionResalte,
        name="Poblacion"
        )

# 4- Añado geometría al mapa
Malagageompobla.add_to(m)

# 5- Le añado la leyenda
m.add_child(escala)

m

### 4- Representación de múltiples capas

Vamos a crear un mapa con un par de capas. Una será el mapa cloroplético que acabamos de hacer, y otra unos marcadores.

In [42]:
from folium import FeatureGroup

# Mapa vacío
m = folium.Map(location = [36.715090, -4.461963],zoom_start= 11)

# Añado los marcadores. 
# Si recuerdas antes los íbamos añadiendo uno a uno al mapa.
# Ahora los iremos añadiendo a un featuregroup, y finalmente este lo añadiremos al mapa. 

# Como ves puedo editar el código html
feature_group = FeatureGroup(name='<span style=\\"color: red;font-size:160%;\\">Marcadores de interés</span>')

folium.Marker(location=[36.714920, -4.477047],
       popup='escuela etsit').add_to(feature_group)

folium.Marker(location=[36.711788, -4.433165],
       popup='María Zambrano').add_to(feature_group)

# Añades marcadores
feature_group.add_to(m)

# Añades geometría clorplética
Malagageompobla.add_to(m)

# Añades la leyenda
m.add_child(escala)

# Añadimos el control de capas
folium.LayerControl().add_to(m)

m